# Import Libraries

In [1]:
import os
from model import Generator, Discriminator
from utils import SaveData, ImagePool
from data import MyDataset
from vgg import Vgg16
from easydict import EasyDict as edict

import torch
import torch.nn as nn
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
from torch.autograd import Variable

from tqdm.notebook import tqdm as tqdm
import time

In [2]:
#Check if gpu is available
torch.cuda.is_available()

True

# Argument Parsing

In [3]:
# from easydict import EasyDict as edict

args = edict()

# Training data
args.dataDir = '/home/trojan/Desktop/Image restoration/Homeworks/HW4/dataset/Dehazing_datasets'  # training data 
args.saveDir = '/home/trojan/Desktop/Image restoration/Homeworks/HW4/result'   # results directory

# Model
args.exp_name = 'Net_dehaze_500'   # model to be selected
args.finetuning = False   # to finetune the training
args.load = None #'NetFinal'

#Validation data
args.val_data = True
args.val_batch_size = 1   # batch size for validation data
#args.n_threads = 8   # threads number for loading data'''

# Testing data
args.testDir = '/home/trojan/Desktop/Image restoration/Homeworks/HW4/dataset/Dehazing_datasets'   # test dataset directory
args.save = True

# Training and Optimization
args.gan_patch = 512   # GAN patch size
args.pool_size = 50   # GAN buffer size
args.batch_size = 1   #training batch size
args.n_threads = 8   # datat laoding number of threads
args.p_factor = 0.5   # perceptual loss factor
args.g_factor = 0.5   # GAN loss factor
args.gen_lr = 1e-4   # generator learning rate
args.disc_lr = 1e-4   #discriminator learning rate
args.lr = 1e-4   # learning rate for the optimizer
args.epochs = 500   # number of training epochs 
args.lr_gamma = 0.5   # learning rate decay factor
args.lr_step_size = 300   # decay learning rate after N epochs
args.decay_type = 'step' #lr decay type

args.period = 1   # period for logging
args.gpu = True   # gpu index

# Device Setup

In [4]:
#Basic Settings
if args.gpu == 0:
    os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
    os.environ["CUDA_VISIBLE_DEVICES"] = "0"
elif args.gpu == 1:
    os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
    os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [5]:
#Check cuda device
device = 'cuda' if torch.cuda.is_available() and args.gpu else 'cpu'
print (device)

cuda


# Training

In [6]:
def train(args):
    print(args)
    
    # net
    netGen = Generator()
    netGen = netGen.cuda()
    netDisc = Discriminator()
    netDisc = netDisc.cuda()
    
    # opt
    optimizerGen = optim.Adam(netGen.parameters(), lr=args.gen_lr)
    optimizerDisc = optim.Adam(netDisc.parameters(), lr=args.disc_lr)

    # lr
    schedulerGen = lr_scheduler.StepLR(optimizerGen, args.lr_step_size, args.lr_gamma)
    schedulerDisc = lr_scheduler.StepLR(optimizerDisc, args.lr_step_size, args.lr_gamma)
    
    # loss
    l1_loss = nn.L1Loss().cuda()
    l2_loss = nn.MSELoss().cuda()
    BCE_loss = nn.BCELoss().cuda()
    
    # models, parameters and logs saving utility
    save = SaveData(args.saveDir, args.exp_name, True)
    save.save_params(args)
    
    # load trainiung data
    dataset = MyDataset(args.dataDir, is_train=True)
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=args.batch_size, shuffle=True,
                                             num_workers=int(args.n_threads))
    
    
    label_real = Variable(torch.ones([1, 1, args.gan_patch, args.gan_patch], dtype=torch.float)).cuda()
    label_fake = Variable(torch.zeros([1, 1, args.gan_patch, args.gan_patch], dtype=torch.float)).cuda()
    
    image_pool = ImagePool(args.pool_size)
    
    vgg = Vgg16(requires_grad=False)
    vgg.cuda()
    
    total_time = 0
    
    for epoch in range(args.epochs):
        start = time.time()
        print("Epoch {}/{}".format(epoch + 1, args.epochs))
        
        schedulerGen.step()
        schedulerDisc.step()
        
        total_real_loss_disc = 0
        total_fake_loss_disc = 0
        total_loss_disc = 0
        
        total_rec_loss_gen = 0
        total_per_loss_gen = 0
        total_gan_loss_gen = 0
        total_gen_loss = 0
        
        netGen.train()
        netDisc.train()
        
        for batch, images in enumerate(dataloader):
            img_input, img_gt = images
            img_input = Variable(img_input.cuda())
            img_gt = Variable(img_gt.cuda())
            img_output = netGen(img_input)
            
            # Discriminator update part
            netDisc.requires_grad(True)
            netDisc.zero_grad()
            
            # Real Image
            real_out = netDisc(img_gt)
            real_loss_disc = BCE_loss(real_out, label_real)
            real_loss_disc.backward()
            real_loss_disc = real_loss_disc.data.cpu().numpy()
            total_real_loss_disc += real_loss_disc
            
            # Fake Image
            fake_img = img_output.detach()
            fake_img = Variable(image_pool.query(fake_img.data))
            fake_out = netDisc(fake_img)
            fake_loss_disc = BCE_loss(fake_out, label_fake)
            fake_loss_disc.backward()
            fake_loss_disc = fake_loss_disc.data.cpu().numpy()
            total_fake_loss_disc += fake_loss_disc
            
            # Total disc loss
            total_loss_disc  += fake_loss_disc + real_loss_disc
            
            optimizerDisc.step()
            
            # Generator update part
            netDisc.requires_grad(False)
            netGen.zero_grad()
            
            # Reconstruction loss calculation
            rec_loss_gen = l1_loss(img_output, img_gt)
            rec_loss_gen.backward(retain_graph=True)
            rec_loss_gen = rec_loss_gen.data.cpu().numpy()
            total_rec_loss_gen += rec_loss_gen
            
            ## perceptual loss
            per_loss_gan = args.p_factor * l2_loss(vgg(img_output), vgg(img_gt))
            per_loss_gan.backward(retain_graph=True)
            per_loss_gan = per_loss_gan.data.cpu().numpy()
            total_per_loss_gen += per_loss_gan
            
            # gan loss
            output = netDisc(img_output)
            gan_loss_gen = args.g_factor * BCE_loss(output, label_real)
            gan_loss_gen.backward()
            gan_loss_gen = gan_loss_gen.data.cpu().numpy()
            total_gan_loss_gen += gan_loss_gen
            
            # Total generator loss
            total_gen_loss += rec_loss_gen + per_loss_gan + gan_loss_gen
            
            optimizerGen.step()
            
        total_real_loss_disc = total_real_loss_disc / (batch + 1)
        total_fake_loss_disc = total_fake_loss_disc / (batch + 1)
        total_loss_disc = total_loss_disc / (batch + 1)
        save.add_scalar('Disc_real', total_real_loss_disc, epoch)
        save.add_scalar('Disc_fake', total_fake_loss_disc , epoch)
        save.add_scalar('Disc_total', total_loss_disc, epoch)

        total_rec_loss_gen = total_rec_loss_gen / (batch + 1)
        total_per_loss_gen = total_per_loss_gen / (batch + 1)
        total_gan_loss_gen = total_gan_loss_gen / (batch + 1)
        total_gen_loss = total_gen_loss / (batch + 1)
        save.add_scalar('Gen_rec', total_rec_loss_gen, epoch)
        save.add_scalar('Gen_per', total_per_loss_gen, epoch)
        save.add_scalar('Gen_gan', total_gan_loss_gen, epoch)
        save.add_scalar('Gen_total', total_gen_loss, epoch)

        end = time.time()
        epoch_time = (end - start)
        total_time += epoch_time
        
        if epoch % args.period == 0:
            log = "Train Discriminator Loss: {:.5f} \t Train Generator Loss_loss: {:.5f} \t Epoch Time: {:.4f} \t Total Time: {:.4f}".format(total_loss_disc, total_gen_loss, epoch_time, total_time)
            print(log)
            save.save_log(log)
            save.save_model(netGen, epoch)

In [7]:
if __name__ == '__main__':
    train(args)

{'dataDir': '/home/trojan/Desktop/Image restoration/Homeworks/HW4/dataset/Dehazing_datasets', 'saveDir': '/home/trojan/Desktop/Image restoration/Homeworks/HW4/result', 'exp_name': 'Net_dehaze_500', 'finetuning': False, 'load': None, 'val_data': True, 'val_batch_size': 1, 'testDir': '/home/trojan/Desktop/Image restoration/Homeworks/HW4/dataset/Dehazing_datasets', 'save': True, 'gan_patch': 512, 'pool_size': 50, 'batch_size': 1, 'n_threads': 8, 'p_factor': 0.5, 'g_factor': 0.5, 'gen_lr': 0.0001, 'disc_lr': 0.0001, 'lr': 0.0001, 'epochs': 500, 'lr_gamma': 0.5, 'lr_step_size': 300, 'decay_type': 'step', 'period': 1, 'gpu': True}
Epoch 1/500


/home/trojan/.local/lib/python3.6/site-packages/torch/optim/lr_scheduler.py:122: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Train Discriminator Loss: 1.38037 	 Train Generator Loss_loss: 2.05218 	 Epoch Time: 77.6715 	 Total Time: 77.6715
Epoch 2/500
Train Discriminator Loss: 1.27781 	 Train Generator Loss_loss: 1.18550 	 Epoch Time: 79.3140 	 Total Time: 156.9855
Epoch 3/500
Train Discriminator Loss: 1.27977 	 Train Generator Loss_loss: 0.97798 	 Epoch Time: 79.7739 	 Total Time: 236.7594
Epoch 4/500
Train Discriminator Loss: 1.32268 	 Train Generator Loss_loss: 0.91963 	 Epoch Time: 80.3958 	 Total Time: 317.1552
Epoch 5/500
Train Discriminator Loss: 1.31376 	 Train Generator Loss_loss: 0.95497 	 Epoch Time: 80.2850 	 Total Time: 397.4402
Epoch 6/500
Train Discriminator Loss: 1.32622 	 Train Generator Loss_loss: 0.92344 	 Epoch Time: 81.0132 	 Total Time: 478.4534
Epoch 7/500
Train Discriminator Loss: 1.33222 	 Train Generator Loss_loss: 0.94458 	 Epoch Time: 81.1885 	 Total Time: 559.6419
Epoch 8/500
Train Discriminator Loss: 1.36872 	 Train Generator Loss_loss: 0.96645 	 Epoch Time: 81.1615 	 Total Time

Epoch 65/500
Train Discriminator Loss: 1.37148 	 Train Generator Loss_loss: 0.87684 	 Epoch Time: 80.4197 	 Total Time: 5246.0373
Epoch 66/500
Train Discriminator Loss: 1.35494 	 Train Generator Loss_loss: 0.86862 	 Epoch Time: 80.7250 	 Total Time: 5326.7623
Epoch 67/500
Train Discriminator Loss: 1.31653 	 Train Generator Loss_loss: 0.88990 	 Epoch Time: 80.9324 	 Total Time: 5407.6947
Epoch 68/500
Train Discriminator Loss: 1.35351 	 Train Generator Loss_loss: 0.94550 	 Epoch Time: 80.8045 	 Total Time: 5488.4993
Epoch 69/500
Train Discriminator Loss: 1.35484 	 Train Generator Loss_loss: 0.94104 	 Epoch Time: 80.8844 	 Total Time: 5569.3837
Epoch 70/500
Train Discriminator Loss: 1.33616 	 Train Generator Loss_loss: 0.88119 	 Epoch Time: 80.7882 	 Total Time: 5650.1719
Epoch 71/500
Train Discriminator Loss: 1.36292 	 Train Generator Loss_loss: 0.90858 	 Epoch Time: 80.9944 	 Total Time: 5731.1662
Epoch 72/500
Train Discriminator Loss: 1.34307 	 Train Generator Loss_loss: 0.90681 	 Epoc

Epoch 128/500
Train Discriminator Loss: 1.35493 	 Train Generator Loss_loss: 0.84476 	 Epoch Time: 81.4314 	 Total Time: 10340.5137
Epoch 129/500
Train Discriminator Loss: 1.34308 	 Train Generator Loss_loss: 0.84594 	 Epoch Time: 81.2160 	 Total Time: 10421.7296
Epoch 130/500
Train Discriminator Loss: 1.33082 	 Train Generator Loss_loss: 0.89441 	 Epoch Time: 81.1746 	 Total Time: 10502.9042
Epoch 131/500
Train Discriminator Loss: 1.33950 	 Train Generator Loss_loss: 0.89463 	 Epoch Time: 80.3494 	 Total Time: 10583.2536
Epoch 132/500
Train Discriminator Loss: 1.36263 	 Train Generator Loss_loss: 0.84300 	 Epoch Time: 80.9033 	 Total Time: 10664.1568
Epoch 133/500
Train Discriminator Loss: 1.34803 	 Train Generator Loss_loss: 0.85842 	 Epoch Time: 80.6583 	 Total Time: 10744.8151
Epoch 134/500
Train Discriminator Loss: 1.31970 	 Train Generator Loss_loss: 0.85831 	 Epoch Time: 80.7281 	 Total Time: 10825.5432
Epoch 135/500
Train Discriminator Loss: 1.35771 	 Train Generator Loss_loss:

Train Discriminator Loss: 1.34478 	 Train Generator Loss_loss: 0.87647 	 Epoch Time: 81.1676 	 Total Time: 15354.2498
Epoch 191/500
Train Discriminator Loss: 1.35863 	 Train Generator Loss_loss: 0.81523 	 Epoch Time: 80.6522 	 Total Time: 15434.9020
Epoch 192/500
Train Discriminator Loss: 1.35322 	 Train Generator Loss_loss: 0.81839 	 Epoch Time: 80.7800 	 Total Time: 15515.6820
Epoch 193/500
Train Discriminator Loss: 1.34918 	 Train Generator Loss_loss: 0.80520 	 Epoch Time: 81.0129 	 Total Time: 15596.6949
Epoch 194/500
Train Discriminator Loss: 1.32656 	 Train Generator Loss_loss: 0.89674 	 Epoch Time: 80.6122 	 Total Time: 15677.3071
Epoch 195/500
Train Discriminator Loss: 1.42117 	 Train Generator Loss_loss: 0.82950 	 Epoch Time: 80.7409 	 Total Time: 15758.0480
Epoch 196/500
Train Discriminator Loss: 1.35478 	 Train Generator Loss_loss: 0.81163 	 Epoch Time: 81.0395 	 Total Time: 15839.0875
Epoch 197/500
Train Discriminator Loss: 1.36246 	 Train Generator Loss_loss: 0.78081 	 Epo

Epoch 253/500
Train Discriminator Loss: 1.36761 	 Train Generator Loss_loss: 0.80247 	 Epoch Time: 81.2331 	 Total Time: 20445.7151
Epoch 254/500
Train Discriminator Loss: 1.34796 	 Train Generator Loss_loss: 0.83844 	 Epoch Time: 80.8649 	 Total Time: 20526.5800
Epoch 255/500
Train Discriminator Loss: 1.34844 	 Train Generator Loss_loss: 0.82529 	 Epoch Time: 81.1166 	 Total Time: 20607.6966
Epoch 256/500
Train Discriminator Loss: 1.34175 	 Train Generator Loss_loss: 0.84029 	 Epoch Time: 81.2049 	 Total Time: 20688.9015
Epoch 257/500
Train Discriminator Loss: 1.38143 	 Train Generator Loss_loss: 0.78621 	 Epoch Time: 80.7482 	 Total Time: 20769.6497
Epoch 258/500
Train Discriminator Loss: 1.34404 	 Train Generator Loss_loss: 0.81991 	 Epoch Time: 80.9812 	 Total Time: 20850.6309
Epoch 259/500
Train Discriminator Loss: 1.35531 	 Train Generator Loss_loss: 0.81412 	 Epoch Time: 81.0083 	 Total Time: 20931.6392
Epoch 260/500
Train Discriminator Loss: 1.36349 	 Train Generator Loss_loss:

Train Discriminator Loss: 1.37634 	 Train Generator Loss_loss: 0.76071 	 Epoch Time: 80.7975 	 Total Time: 25463.7096
Epoch 316/500
Train Discriminator Loss: 1.34341 	 Train Generator Loss_loss: 0.76846 	 Epoch Time: 80.9813 	 Total Time: 25544.6909
Epoch 317/500
Train Discriminator Loss: 1.38814 	 Train Generator Loss_loss: 0.77236 	 Epoch Time: 80.6102 	 Total Time: 25625.3010
Epoch 318/500
Train Discriminator Loss: 1.36288 	 Train Generator Loss_loss: 0.77367 	 Epoch Time: 80.8476 	 Total Time: 25706.1486
Epoch 319/500
Train Discriminator Loss: 1.37111 	 Train Generator Loss_loss: 0.77284 	 Epoch Time: 80.8460 	 Total Time: 25786.9947
Epoch 320/500
Train Discriminator Loss: 1.35417 	 Train Generator Loss_loss: 0.77457 	 Epoch Time: 81.2885 	 Total Time: 25868.2831
Epoch 321/500
Train Discriminator Loss: 1.34790 	 Train Generator Loss_loss: 0.78202 	 Epoch Time: 81.0110 	 Total Time: 25949.2942
Epoch 322/500
Train Discriminator Loss: 1.33721 	 Train Generator Loss_loss: 0.73711 	 Epo

Epoch 378/500
Train Discriminator Loss: 1.36442 	 Train Generator Loss_loss: 0.72346 	 Epoch Time: 80.9428 	 Total Time: 30563.9086
Epoch 379/500
Train Discriminator Loss: 1.37416 	 Train Generator Loss_loss: 0.75646 	 Epoch Time: 81.1062 	 Total Time: 30645.0149
Epoch 380/500
Train Discriminator Loss: 1.35733 	 Train Generator Loss_loss: 0.77055 	 Epoch Time: 81.1606 	 Total Time: 30726.1755
Epoch 381/500
Train Discriminator Loss: 1.36872 	 Train Generator Loss_loss: 0.76928 	 Epoch Time: 80.8480 	 Total Time: 30807.0235
Epoch 382/500
Train Discriminator Loss: 1.34334 	 Train Generator Loss_loss: 0.75355 	 Epoch Time: 81.0762 	 Total Time: 30888.0997
Epoch 383/500
Train Discriminator Loss: 1.37268 	 Train Generator Loss_loss: 0.78890 	 Epoch Time: 81.0036 	 Total Time: 30969.1033
Epoch 384/500
Train Discriminator Loss: 1.35386 	 Train Generator Loss_loss: 0.77725 	 Epoch Time: 80.9254 	 Total Time: 31050.0287
Epoch 385/500
Train Discriminator Loss: 1.36231 	 Train Generator Loss_loss:

Train Discriminator Loss: 1.36562 	 Train Generator Loss_loss: 0.73412 	 Epoch Time: 81.0318 	 Total Time: 35595.2324
Epoch 441/500
Train Discriminator Loss: 1.34635 	 Train Generator Loss_loss: 0.71808 	 Epoch Time: 80.6820 	 Total Time: 35675.9144
Epoch 442/500
Train Discriminator Loss: 1.35543 	 Train Generator Loss_loss: 0.74667 	 Epoch Time: 81.4127 	 Total Time: 35757.3271
Epoch 443/500
Train Discriminator Loss: 1.36166 	 Train Generator Loss_loss: 0.76196 	 Epoch Time: 81.1402 	 Total Time: 35838.4673
Epoch 444/500
Train Discriminator Loss: 1.36132 	 Train Generator Loss_loss: 0.78013 	 Epoch Time: 80.7939 	 Total Time: 35919.2612
Epoch 445/500
Train Discriminator Loss: 1.36652 	 Train Generator Loss_loss: 0.75519 	 Epoch Time: 81.1233 	 Total Time: 36000.3845
Epoch 446/500
Train Discriminator Loss: 1.34947 	 Train Generator Loss_loss: 0.75648 	 Epoch Time: 81.3981 	 Total Time: 36081.7826
Epoch 447/500
Train Discriminator Loss: 1.34548 	 Train Generator Loss_loss: 0.74095 	 Epo